In [10]:
import numpy as np
import soundfile as sf

# Set the desired duration of the meditation music in seconds
desired_duration = 22 * 60

# Set the sample rate
sample_rate = 44100

# Define the frequency range
min_frequency = 174
max_frequency = 286

# Define the segment duration range
min_segment_duration = 15
max_segment_duration = 45

# Set the ramp up duration at the beginning in seconds
ramp_up_duration = 5

# Load water sound, bird sound, and string instrument layers
water_sound, _ = sf.read('water_sound.wav')
bird_sound, _ = sf.read('bird_sound.wav')
string_layers = [
    sf.read('string_layer1.wav')[0],
    sf.read('string_layer2.wav')[0],
    sf.read('string_layer3.wav')[0]
]

# Calculate the number of samples for the desired duration
desired_num_samples = int(desired_duration * sample_rate)

# Generate the meditation music
samples = np.zeros(desired_num_samples)

current_time = 0
while current_time < desired_duration:
    # Generate random segment duration
    segment_duration = np.random.randint(min_segment_duration, max_segment_duration + 1)

    # Generate random frequency for the segment
    frequency = np.random.uniform(min_frequency, max_frequency)

    # Calculate the number of samples for the segment
    segment_num_samples = int(segment_duration * sample_rate)

    # Generate the segment samples with ramp up and ramp down
    segment_time = np.arange(segment_num_samples) / sample_rate
    segment_samples = np.sin(2 * np.pi * frequency * segment_time)

    # Apply volume ramp up at the beginning of the segment
    ramp_up_samples = np.linspace(0, segment_samples[0], int(ramp_up_duration * sample_rate))
    segment_samples[:len(ramp_up_samples)] *= ramp_up_samples

    # Apply volume ramp down at the end of the segment
    ramp_down_samples = np.linspace(segment_samples[-1], 0, int(ramp_up_duration * sample_rate))
    segment_samples[-len(ramp_down_samples):] *= ramp_down_samples

    # Resize water sound to match segment samples length
    water_samples = np.resize(water_sound, segment_num_samples)
    if len(water_samples) != len(segment_samples):
        water_samples = np.resize(water_sound, len(segment_samples))
    # Add water sound
    segment_samples += water_samples

    # Resize bird sound to match segment samples length
    bird_samples = np.resize(bird_sound, segment_num_samples)
    if len(bird_samples) != len(segment_samples):
        bird_samples = np.resize(bird_sound, len(segment_samples))
    # Add bird sound
    segment_samples += bird_samples

    # Add layered string instrument sounds
    string_samples = np.zeros(segment_num_samples)
    for layer in string_layers:
        layer_samples = np.resize(layer, segment_num_samples)
        if len(layer_samples) != len(segment_samples):
            layer_samples = np.resize(layer, len(segment_samples))
        string_samples += layer_samples

    # Add string instrument sounds
    segment_samples += string_samples

    # Concatenate the segment samples to the overall samples
    start_sample = int(current_time * sample_rate)
    end_sample = start_sample + len(segment_samples)

    # Adjust the sizes of the arrays for concatenation
    if end_sample > desired_num_samples:
        segment_samples = segment_samples[:desired_num_samples - start_sample]
        end_sample = desired_num_samples

    samples[start_sample:end_sample] += segment_samples

    # Update the current time
    current_time += segment_duration

# Normalize the samples
samples /= np.max(np.abs(samples))

# Save the generated audio to a WAV file
output_file = "meditation_music.wav"
sf.write(output_file, samples, sample_rate)

print(f"Meditation music generated and saved as {output_file}.")


Meditation music generated and saved as meditation_music.wav.
